<a href="https://colab.research.google.com/github/galrat/parsing/blob/main/!patent_purity.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# установка пакетов

In [1]:
# установка пакетов
!pip install selenium
!pip install langid
from bs4 import BeautifulSoup
from google.colab import files

from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as ec
from selenium.webdriver.support.ui import WebDriverWait

import pandas as pd
import requests
import time

import re
import langid

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.5/10.5 MB 16.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 467.2/467.2 kB 22.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 4.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 9.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for langid: filename=langid-1.1.6-py3-none-any.whl size=1941172 sha256=2ea08b1b6ef0f7ec2da8ec71a991b8fce5a7c9741a2ebf2e1deccf6413b39ef7
  Stored in directory: /root/.cache/pip/wheels/23/c8/c6/eed80894918490a175677414d40bd7c851413bbe03d4856c3c
Successfully built langid


In [2]:
target_country = 'US'
start_year = '2000-01-01'
title_filter_presence = []
title_filter_absence =  []


# Что нужно сделать для получения таблицы с патентами:
1. ниже надо ввести номера патентов для парсинга
2. затем нажать ctrl+F9 или выбрать Runtime->Run all в меню сверху

In [31]:
# original numbers for parsing

numbers = '''JP3699345B2
US7090391B2
CA2461006C
US7331702B2
WO2009102955A2
US7762340B2
CN101829515A
JP4868656B2
US10683397B2
US8215930B2
CA2741849C
CN202805411U
CN203196585U
CA2737377C
CA2871588A1
CA2700239C
KR101718379B1
KR101574578B1
CN205020005U
CN205997178U
US9903169B2
CN208812341U
CN110052209A
CN210065212U
US20210048149A1
KR102270569B1
CN213726112U
CN213824570U
CN213854408U
CN110015736B
CN109395619B
CN215353214U
CA3090883C
CA2938116C
CN114392679A
CN216879092U
CN111074714B
CN217163999U
CA3209186A1
CA3041936C
CA3076363A1
CA3076363C
WO2023132877A1
CN219522752U
CA3029476C
CN112537005A
CA3019370C
'''


# основной код

## defs

In [32]:
def separate_languages(text):
    sentences = text.split('.')  # You can use a more sophisticated sentence tokenizer based on your text

    english_text = []
    other_languages_text = []

    for sentence in sentences:
        # Use langid to identify the language of each sentence
        lang, confidence = langid.classify(sentence)

        if lang == 'en':
            english_text.append(sentence)
        else:
            other_languages_text.append(sentence)

    return ' '.join(english_text)

In [33]:
def get_claim_google(soup):
  all = soup.find_all('div', class_='flex flex-width style-scope patent-result')[-1].find_all('div', class_='claim-text style-scope patent-text')[1]
  return all.text

def get_claim_google_2(soup):
  try:
      # all
      try:
          try:
              all = soup.find('div', class_='claim-text style-scope patent-text').find('span', class_='notranslate style-scope patent-text').text
          except:
              all = soup.find('div', class_='claim-text style-scope patent-text').text
          #print('all', all)
      except:
          all = soup.find('section', id='claims').find('span', class_='notranslate style-scope patent-text').text
          #print('all', all)

      # original
      try:
          original = soup.find('div', class_='claim-text style-scope patent-text').find('span',
                                                                                        class_='notranslate style-scope patent-text').find(
              'span').text
      except:
          try:
              original = soup.find('section', id='claims').find('span',
                                                                class_='notranslate style-scope patent-text').find(
                  'span').text
          except:
              original = ''
      #print('original', original)

      # translation
      claim_1 = all.replace(original, '')
      #print('claim 1:', claim_1)
  except:
      claim_1 = 'no_data'
  if claim_1 == '' or claim_1 == 'no_data':
    try:
      claim_1 = soup.find_all('div', class_='flex flex-width style-scope patent-result')[-1].find_all('div', class_='claim-text style-scope patent-text')[1].text
    except:
      claim_1 = 'no_data'
  return claim_1

In [34]:
def get_data_from_google_patent(url, target_country):
  options = Options()
  options.add_argument("--headless")
  options.add_argument('--no-sandbox')
  options.add_argument('--user-agent=Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/92.0.4515.131 Safari/537.36')
  driver = webdriver.Chrome(options=options)
  driver.get(url)
  WebDriverWait(driver, 20).until(ec.visibility_of_element_located((By.ID, 'content')))
  soup = BeautifulSoup(driver.page_source, 'lxml')

  # find US, EP, WO patent document

  others = soup.find('div', class_='event style-scope application-timeline')
  text = others.find_all(class_='style-scope application-timeline')[0]
  other_apps = ''
  for t in others.find_all(class_='style-scope application-timeline'):
      try:
        other_apps += t.get('data-result').split('patent/')[1].split('/')[0] + ','
      except:
        other_apps = other_apps
  print('other_apps', other_apps)

  if target_country not in url:
    new_app = ''
    for app_number in other_apps.split(','):
      #print('app_number', app_number)
      if 'US' in app_number:
        new_app = app_number
        break
      if 'EP' in app_number:
        new_app = app_number
        #break
      if 'WO' in app_number:
        new_app = app_number
        #break
    if new_app != '':
      url = 'https://patents.google.com/patent/' + new_app + '/en'
      print('new_url', url)
      options = Options()
      options.add_argument("--headless")
      options.add_argument('--no-sandbox')
      options.add_argument('--user-agent=Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/92.0.4515.131 Safari/537.36')
      driver = webdriver.Chrome(options=options)
      driver.get(url)
      WebDriverWait(driver, 20).until(ec.visibility_of_element_located((By.ID, 'content')))
      soup = BeautifulSoup(driver.page_source, 'lxml')
    else:
      print('url is not changed since no US, EP, WO is detected', url)
  else:
    print('url is not changed since it is target')

  # title
  title = soup.find('div', id='wrapper').find('h1', id='title').text.replace('\n', '').strip()
  #print('title:', title)

  # patent_number
  patent_number = soup.find('h2', id='pubnum').text.strip().replace('\n', '').replace('\t', '')
  #print('patent_number:', patent_number)

  # applicant
  author_counter = 0
  applicant_counter = 0
  counter = 0
  author_check = 0
  applicant_check = 0
  important_people_data = soup.find('dl', class_='important-people style-scope patent-result')
  for i in important_people_data.find_all('dt', class_='style-scope patent-result'):
      if 'Inventor' in i.text:
          author_counter = counter
          author_check = 1
      if 'assignee' in i.text:
          applicant_counter = counter
          applicant_check = 1
      counter += 1
  #print('author_counter', author_counter, 'applicant_counter', applicant_counter)

  authors = ''
  if author_check == 1:
      try:
          authors_data = soup.find('dl', class_='important-people style-scope patent-result')
          authors_data = authors_data.find_all('dt', class_='style-scope patent-result')[
              author_counter].find_next_siblings('dd')
          for author in authors_data:
              if '\n' in author.text:
                  break
              authors = authors + author.text + ';'
          authors = authors.replace('\n', '')
      except:
          #print('no authors data')
          authors = ('no authors data')
  else:
      authors = 'no_data'

  applicants = ''
  if applicant_check == 1:
      try:
          applicants_data = soup.find('dl', class_='important-people style-scope patent-result')
          applicants_data = applicants_data.find_all('dt', class_='style-scope patent-result')[
              applicant_counter].find_next_siblings('dd')
          for applicant in applicants_data:
              applicants = applicants + applicant.text.strip() + ';'
          applicants = applicants.replace('\n', '').strip()
      except:
          #print('no applicants data')
          applicants = ('no applicants data')
  else:
      applicants = 'no_data'

  # filing_date
  filind_date = 'no_data'
  publication_date = 'no_data'
  status = 'no_data'
  dates_data = soup.find_all('div', class_='event layout horizontal style-scope application-timeline')
  for data in dates_data:
      if 'filed by' in data.text:
          filind_date = data.text.split('Application')[0]
          #print('filing date:', filind_date)
      if 'Publication' in data.text:
          publication_date = data.text.split('Publication')[0]
          #print('publication date:', publication_date)
      if 'Status' in  data.text:
          status = data.text.split('Status')[1].replace('\n', '').replace('\t', '')
          #print('Status:', status)

  # application number
  try:
      application_number = soup.find_all('div', class_='header style-scope application-timeline')[1].text
      application_number = application_number.split('Application')[1].split('events')[0].replace(' ','')
  except:
      print('no application number data')
      application_number = 'no data'
  #print('application_number', application_number)

  # get priority applications
  priority_applications = ''
  for i in soup.find('div', class_='wrap style-scope application-timeline').find_all('div', class_='event layout horizontal style-scope application-timeline'):
    try:
      priority_applications += i.text.split('Priority to')[1].strip() + ';'
      #print(i.text.split('Priority to')[1].strip())
    except:
      no_priority = 1
  if priority_applications == '':
    priority_applications = 'no_data'

  # ipc
  ipc_data = 'no_data'
  try:
      ipcs = soup.find('div', class_='style-scope classification-viewer').find_all('div', class_='style-scope classification-tree')
      for ipc in ipcs:
          ipc_class = ipc.find(first=True).text.strip().replace('\n', '').replace('\t', '')
          #print('ipc', ipc_class)
      #print(ipc_class)
  except:
      ipc_data = 'no_data'

  ipc_datas = soup.find_all('div', class_='style-scope classification-tree')
  for i in ipc_datas[:1]:
      target = i.find_all(class_='code style-scope classification-tree')
      #print('target ipc', target[-1].text)
      # print(i.text.strip())
      ipc_data = target[-1].text

  # claim 1
  claim_1 = ''
  try:
    claim_1 = get_claim_google_2(soup)
    #print('claim_1 google', claim_1)
  except:
    print('no claims')
  claim_1 = claim_1.split('according to claim')[0].replace('\n', '').replace('\t', ' ')

  # abstract
  abstract = ''
  try:
    abstract_orig = soup.find('section', id='abstract').find('span', class_='google-src-text style-scope patent-text').text#.text.replace('\n', '').replace('\t', '').replace('Abstracttranslated from ', '')
    #print('abstract_orig', abstract_orig)
    abstract_all = soup.find('section', id='abstract').text
    #print('abstract_all', abstract_all)
    abstract_eng = abstract_all.replace(abstract_orig, '').strip()
    abstract_eng = abstract_eng.replace('\n', '')
    #print('abstract_eng', abstract_eng)
    abstract = abstract_eng
  except:
    abstract = soup.find('section', id='abstract').text.replace('\n', '').replace('\t', ' ')

  # set 'printing = 1' to show info
  printing = 0
  while printing:
      print('patent_number:', patent_number)
      print('application_number:', application_number)
      print('title:', title)
      print('priority_applications:', priority_applications)
      print('filind_date:', filind_date)
      print('applicant:', applicants)
      print('authors:', authors)
      print('ipc_data:', ipc_data)
      print('publication_date:', publication_date)
      print('abstract:', abstract)
      print('claim_1:', claim_1)
      print('status:', status)
      print('others:', other_apps)
      break

  header = ['patent number', 'application_number', 'title', 'INV/UM', 'filing_date', 'applicant', 'autors',
                          'publiction_date', 'ipc', 'claims', 'status', 'abstract', 'other_applications']
  data_line = []
  data_line.append(patent_number)
  data_line.append(application_number)
  data_line.append(title)
  data_line.append(priority_applications)
  data_line.append(filind_date)
  data_line.append(applicants)
  data_line.append(authors)
  data_line.append(publication_date)
  data_line.append(ipc_data)
  data_line.append(claim_1.replace('\n', ''))
  data_line.append(status)
  data_line.append(abstract.replace('\n', ''))
  #data_line.append(others[:-1])
  data_line.append(other_apps)

  # =========================================================================================
  # data from the bottom reagrding similar patents

  add_patents = soup.find('div', class_='footer style-scope patent-result').find_all('div', class_='responsive-table style-scope patent-result')[0].find_all('div', class_='tr style-scope patent-result')
  patent_lines = []
  for patent in add_patents:
    patent_line = []
    for line in patent.find_all('span'):
      #print(line.text.replace('*', '').strip())
      patent_line.append(line.text.replace('*', '').strip())
      #print('---------------------')
    #print(patent.find('span'))
    patent_lines.append(patent_line)
    #print('===========================')
    #break

  add_patents = soup.find('div', class_='footer style-scope patent-result').find_all('div', class_='responsive-table style-scope patent-result')[1].find_all('div', class_='tr style-scope patent-result')
  for patent in add_patents:
    patent_line = []
    for line in patent.find_all('span'):
      #print(line.text.replace('*', '').strip())
      patent_line.append(line.text.replace('*', '').strip())
      #print('---------------------')
    #print(patent.find('span'))
    if len(patent_line) == 3:
      patent_line.insert(1, 'no_data')
      patent_line.insert(3, 'no_data')
    patent_lines.append(patent_line)
    #print('===========================')
    #break

  add_patents = soup.find('div', class_='footer style-scope patent-result').find_all('div', class_='responsive-table style-scope patent-result')[2].find_all('div', class_='tr style-scope patent-result')
  for patent in add_patents:
    patent_line = []
    for line in patent.find_all('span'):
      #print(line.text.replace('*', '').strip())
      patent_line.append(line.text.replace('*', '').strip())

      #print('---------------------')
    #print(patent.find('span'))
    if len(patent_line) == 3:
      patent_line.insert(1, 'no_data')
      patent_line.insert(3, 'no_data')
    patent_lines.append(patent_line)
    #print('===========================')
    #break

  # get data for espacenet
  all_additional_urls = [url.get('href') for url in soup.find('dl', class_='links style-scope patent-result').findAll('a')]
  bottom_data = patent_lines

  return data_line, bottom_data, all_additional_urls


In [35]:
def get_from_espacenet(espacenet_url):
    #print('def', espacenet_url)
    data_line2 = []

    options = Options()
    options.add_argument("--headless")
    options.add_argument('--no-sandbox')
    options.add_argument('--user-agent=Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/92.0.4515.131 Safari/537.36')
    driver = webdriver.Chrome(options=options)
    driver.get(espacenet_url)
    time.sleep(2)
    WebDriverWait(driver, 20).until(ec.visibility_of_element_located((By.ID, 'pagebody')))

    soup = BeautifulSoup(driver.page_source, 'lxml')

    #print(soup.text)
    #print('---------------------\n')
    # title
    title = soup.find('h3').text.strip().replace('\n', '').replace('\t', '')
    #print('title:', title)
    table = soup.find('table', class_='tableType3')
    #print(table.text.strip())
    app_num = soup.find('h1', class_='noBottomMargin').text.split('―')[0].split('\n')[-1].strip().replace('\xa0', '').replace('(', '').replace(')', '')

    applicant = ''
    authors = ''
    ipc = ''
    priority = ''
    original_application = ''
    others= ''

    table_lines = table.find_all('tr')
    for line in table_lines:
      if 'Applicant(s):' in line.text:
        applicant = ' '.join(line.find('td').text.replace('\n', '').replace('\t', '').strip().split())
        applicant = applicant.split('+')[0].strip()
        #print('applicant:', applicant)

      if 'Inventor(s):' in line.text:
        authors = ' '.join(line.find('td').text.replace('\n', '').replace('\t', '').strip().split())
        authors = authors.split('+')[0].strip()
        #print('authors:', authors)

      if 'Classification:' in line.text:
        ipc = ' '.join(line.find('td').text.replace('\n', '').replace('\t', '').strip().split()).split('- cooperative')[0]
        ipc = ipc.replace('- international:', '').strip()
        ipc = ipc.split('(IPC1-7)')[0].strip()
        #print('ipc:', ipc)

      if 'Priority number(s):' in line.text:
        priority = ' '.join(line.find('td').text.replace('\n', '').replace('\t', '').strip().split()).split('- cooperative')[0]
        #print('Priority:', priority)

      if 'Application number:' in line.text:
        original_application = ' '.join(line.find('td').text.replace('\n', '').replace('\t', '').strip().split()).split('- cooperative')[0]
        original_application = original_application.replace('Global Dossier', '').strip()
        #print('original_application:', original_application)

      if 'Also published as:' in line.text:
        others = ' '.join(line.find('div', id='lessPublishedAs').text.replace('\n', '').replace('\t', '').strip().split()).split('- cooperative')[0]
        #others = soup.find('div', id='lessPublishedAs').text.strip()
        #print('others:', others)


    publication = 'no_data'

    abstract = soup.find('div', class_='application article clearfix').find('p').text.replace('\t', ' ').replace('\n', '')
    #print(abstract)

    printing = 0
    while printing:
        print('title:', title)
        print('applicant:', applicant)
        print('authors:', authors)
        print('ipc:', ipc)
        print('priority:', priority)
        print('original_application:', original_application)
        print('publication:', publication)
        print('abstract:', abstract)
        print('others:', others)
        break

    data_line2.append(app_num)
    data_line2.append(applicant)
    data_line2.append(authors)
    data_line2.append(ipc)
    data_line2.append(priority)
    data_line2.append(original_application)
    data_line2.append(publication)
    data_line2.append(title)
    data_line2.append(abstract)
    data_line2.append(others)

    header = ['patent_number2', 'applicant2', 'authors2', 'ipc2', 'priority2', 'original2', 'publication2', 'title', 'abstract', 'other_applications']

    return (data_line2)

In [36]:
def get_patent_data(numbers, bottom_data_file):
  numbers = numbers.replace('-', '')
  all_numbers = numbers.split()
  print('lenght of all_numbers', len(all_numbers))

  # проверка, какие номера патентов еще не спарсены
  file_for_saving_results = "/content/saved_data.txt"

  header_1 = ['patent number', 'applicant2', 'authors2', 'ipc2', 'priority2', 'original2', 'publication2', 'title_2', 'abstract_2',
                'other_applications_2']
  header_2 = ['patent number', 'application_number', 'title', 'priority applications', 'filing_date', 'applicant', 'authors',
                              'publiction_date', 'ipc', 'claims', 'status', 'abstract', 'other_applications']
  try:
      saved_data = pd.read_csv(file_for_saving_results, sep='\t')
      parsed_numbers = saved_data['original_number'].values
      print('parsed_numbers', parsed_numbers)
  except:
      parsed_numbers = []
      with open(file_for_saving_results, 'a', encoding="utf-8") as file:
          file.writelines('\t'.join(['original_number'] + header_2 + header_1) + '\n')
      print("creating new file")

  remaining_numbers = list(set(all_numbers) - set(parsed_numbers))
  print('remaining_numbers', remaining_numbers)
  print('total remaining', len(remaining_numbers))

  # correct app_numbers
  remaining_numbers_corrected = []
  for number in remaining_numbers:
      if number[:2] == 'WO' and len(number) == 12:
          number = 'WO20' + number[2:] + '/en'
      remaining_numbers_corrected.append(number)

  final_urls = []
  for app_num in remaining_numbers_corrected:
    if app_num[:3] == 'US2' and len(app_num) == 14:
        url =  app_num[:].replace('-', '')[:6] + '0' + app_num[:].replace('-', '')[6:] + '/en'
        url = 'https://patents.google.com/patent/' + url
        #print('corrected url', url)
    else:
        url = 'https://patents.google.com/patent/' + app_num[:].replace('-', '').replace('\n', '') + '/en'
        #print('url', url)
    final_urls.append(url)

  print(final_urls)

  # результаты

  # get data by url
  counter = 1
  wrong_numbers = []
  bottom_data = []
  espacenet_urls = []
  for original_number, url in zip(remaining_numbers, final_urls[:]):
    print(counter, '/', len(remaining_numbers))
    counter += 1
    print(url)
    try:
      data_line_google = get_data_from_google_patent(url, target_country)
      #print('data_line_google', data_line_google[0])
      #print(data_line_google[1])
      bottom_data += data_line_google[1]

      # get espacenet url
      all_additional_urls = data_line_google[2]
      for url in all_additional_urls:
        if 'espacenet' in url:
          #print('espacenet_url from google', url)
          espacenet_url = url
          espacenet_urls.append(espacenet_url)
          break

      # get data using espacent url
      try:
        o/0
        print('espacenet_url', espacenet_url)
        data_line_espacenet = get_from_espacenet(espacenet_url)
        print('data_line_espacenet', data_line_espacenet)
        print()
      except:
        data_line_espacenet = header_1
        print('smth wrong with espacenet')

      with open(file_for_saving_results, 'a', encoding="utf-8") as file:
          file.writelines('\t'.join([original_number] + data_line_google[0] + data_line_espacenet) + '\n')
          print('writing')
    except:
      wrong_numbers.append(original_number)



    print()
    #break
  print('====================finish=========================')
  print('wrong_numbers which shall be reparsed', wrong_numbers)

  # download the created file
  #files.download(file_for_saving_results)
  bottom_data_2 = []

  for line in bottom_data[1:]:
    if len(line) == 5 and 'Family To Family Citations' not in line[0] and 'Priority date' not in line[1]:
      bottom_data_2.append(line)

  bottom_data_df = pd.DataFrame(bottom_data_2[0:], columns=['publication number', 'Priority date', 'publication date',  'assignee',  'title'])
  bottom_data_df.to_csv(bottom_data_file)
  #print(bottom_data_df)
  #files.download('bottom_data.csv')



## code

In [37]:
# первый этап, на котором парсятся номера патентов и собираются данные из bottom part (cited, citing, similar)
bottom_data_file = '/content/bottom_data_step_1.txt'
get_patent_data(numbers, bottom_data_file)

first_step_data = pd.read_csv('/content/saved_data.txt', sep='\t')
print('количество документов на первом этапе', len(first_step_data))
already_parsed = set(first_step_data['patent number'].values)
#already_parsed


lenght of all_numbers 47
parsed_numbers ['CN205020005U' 'US10683397B2' 'CA3076363A1' 'CN208812341U'
 'WO2009102955A2' 'CN219522752U' 'CN111074714B' 'US8215930B2'
 'CN110015736B' 'JPH08266880A' 'CA3076363C' 'CN213726112U' 'KR102270569B1'
 'CN213854408U' 'CN109395619B' 'US20210048149A1' 'CN217163999U'
 'CN213824570U' 'CN210065212U' 'CN110052209A' 'WO2023132877A1'
 'CN112537005A' 'US7331702B2' 'CN114392679A' 'KR101718379B1' 'US9903169B2'
 'CN101829515A' 'CN215353214U' 'KR101574578B1' 'CN203196585U'
 'JP3699345B2' 'US7762340B2' 'CA2741849C' 'CN216879092U' 'JP4868656B2'
 'CN202805411U' 'CN205997178U' 'US7090391B2' 'CA2737377C' 'CA2700239C'
 'CA3029476C' 'CA3019370C' 'CA3090883C' 'CA3209186A1' 'CA3041936C'
 'CA2461006C' 'CA2938116C' 'CA2871588A1']
remaining_numbers []
total remaining 0
[]
====================finish=========================
wrong_numbers which shall be reparsed []
количество документов на первом этапе 48


In [38]:
# обработка bottom file
df = pd.read_csv(bottom_data_file, sep=',', index_col='Unnamed: 0')
print('исходное количество bottom patents:', len(df))

df = df.drop_duplicates(['publication number'], keep='first')
df = df.dropna()
filtered_df = df[~df['publication number'].isin(already_parsed)]
print('первый этап фильтрации (drop_duplicates, dropna, drop already parsed)', len(filtered_df))

number_stop_words = [' et al.'] # to delete articles and save patents
filtered_df = filtered_df[filtered_df['title'].str.contains('|'.join(title_filter_presence), case=False) & ~filtered_df['title'].str.contains('|'.join(title_filter_absence), case=False)]
filtered_df = filtered_df[~filtered_df['publication number'].str.contains('|'.join(number_stop_words), case=False)]
print('второй этап фильтрации (по ключевым словам в названии)', len(filtered_df))
#print(filtered_df[['publication number', 'title']])
print('отфильтрованы (исключены)', set(df['publication number'].values) - set(filtered_df['publication number'].values))
print()

pattern = r'^[A-Z]{2}'
filtered_df = filtered_df[filtered_df['publication number'].str.match(pattern)]

filtered_df['publication date'] = pd.to_datetime(filtered_df['publication date'])
filtered_df = filtered_df[filtered_df['publication date'] > start_year]
filtered_df = filtered_df[filtered_df['publication number'].str.contains(target_country)]

filtered_df = filtered_df.sort_values('title', ascending=True)
#for i in range(len(filtered_df)):
  #print(filtered_df.iloc[i][['title']].values)

second_step_numbers = '\n'.join(filtered_df['publication number'].values)
print('третий этап фильтрации (только целевая страна, только подходящая дата)', len(filtered_df['publication number'].values), '\n', second_step_numbers)


исходное количество bottom patents: 0
первый этап фильтрации (drop_duplicates, dropna, drop already parsed) 0
второй этап фильтрации (по ключевым словам в названии) 0
отфильтрованы (исключены) set()

третий этап фильтрации (только целевая страна, только подходящая дата) 0 
 


In [39]:
# второй этап, на котором парсятся данные по номерам патентов из bottom part после первого парсинга

bottom_data_file_empty = '/content/bottom_data_step_empty.txt' # второй bottom data файл не используется, но его нужно указать

get_patent_data(second_step_numbers, bottom_data_file_empty)

lenght of all_numbers 0
parsed_numbers ['CN205020005U' 'US10683397B2' 'CA3076363A1' 'CN208812341U'
 'WO2009102955A2' 'CN219522752U' 'CN111074714B' 'US8215930B2'
 'CN110015736B' 'JPH08266880A' 'CA3076363C' 'CN213726112U' 'KR102270569B1'
 'CN213854408U' 'CN109395619B' 'US20210048149A1' 'CN217163999U'
 'CN213824570U' 'CN210065212U' 'CN110052209A' 'WO2023132877A1'
 'CN112537005A' 'US7331702B2' 'CN114392679A' 'KR101718379B1' 'US9903169B2'
 'CN101829515A' 'CN215353214U' 'KR101574578B1' 'CN203196585U'
 'JP3699345B2' 'US7762340B2' 'CA2741849C' 'CN216879092U' 'JP4868656B2'
 'CN202805411U' 'CN205997178U' 'US7090391B2' 'CA2737377C' 'CA2700239C'
 'CA3029476C' 'CA3019370C' 'CA3090883C' 'CA3209186A1' 'CA3041936C'
 'CA2461006C' 'CA2938116C' 'CA2871588A1']
remaining_numbers []
total remaining 0
[]
====================finish=========================
wrong_numbers which shall be reparsed []


In [40]:
# бывает, что имеется множество US заявок, для исследования чистоты надо их все собрать, здесь я нахожу их номера
second_step_data = pd.read_csv('/content/saved_data.txt', sep='\t')
third_step_numbers = []
for line in second_step_data['other_applications']:
  for i in line.split(','):
    if target_country in i:
      third_step_numbers.append(i)

third_step_numbers = set(third_step_numbers) - set(second_step_data['patent number'].values)

third_step_numbers = '\n'.join(third_step_numbers)
print(third_step_numbers)

In [41]:
# здесь парсятся дополнительные номера заявок из other_applications
bottom_data_file_empty = '/content/bottom_data_step_empty.txt'

get_patent_data(third_step_numbers, bottom_data_file_empty)

lenght of all_numbers 0
parsed_numbers ['CN205020005U' 'US10683397B2' 'CA3076363A1' 'CN208812341U'
 'WO2009102955A2' 'CN219522752U' 'CN111074714B' 'US8215930B2'
 'CN110015736B' 'JPH08266880A' 'CA3076363C' 'CN213726112U' 'KR102270569B1'
 'CN213854408U' 'CN109395619B' 'US20210048149A1' 'CN217163999U'
 'CN213824570U' 'CN210065212U' 'CN110052209A' 'WO2023132877A1'
 'CN112537005A' 'US7331702B2' 'CN114392679A' 'KR101718379B1' 'US9903169B2'
 'CN101829515A' 'CN215353214U' 'KR101574578B1' 'CN203196585U'
 'JP3699345B2' 'US7762340B2' 'CA2741849C' 'CN216879092U' 'JP4868656B2'
 'CN202805411U' 'CN205997178U' 'US7090391B2' 'CA2737377C' 'CA2700239C'
 'CA3029476C' 'CA3019370C' 'CA3090883C' 'CA3209186A1' 'CA3041936C'
 'CA2461006C' 'CA2938116C' 'CA2871588A1']
remaining_numbers []
total remaining 0
[]
====================finish=========================
wrong_numbers which shall be reparsed []


In [42]:
final_data = pd.read_csv('/content/saved_data.txt', sep='\t')
final_data

,original_number,patent number,application_number,title,priority applications,filing_date,applicant,authors,publiction_date,ipc,...,patent number.1,applicant2,authors2,ipc2,priority2,original2,publication2,title_2,abstract_2,other_applications_2
0,CN205020005U,CN205020005U,CN201520753612.8U,"Resin, curing agent mixing stirring device",CN201520753612.8U;,2015-09-25,Beijing Chaobai Environmental Protection Equip...,国春勇;,2016-02-10,no_data,...,patent number,applicant2,authors2,ipc2,priority2,original2,publication2,title_2,abstract_2,other_applications_2
1,US10683397B2,US20120035297A1,"US13/194,409",Remediation of agglomerated flow improvers,"US13/194,409;",2011-07-29,Lubrizol Specialty Products Inc;,"William Franklin Harris, Jr.;Ray Lyle Johnston;",2012-02-09,C08J3/12,...,patent number,applicant2,authors2,ipc2,priority2,original2,publication2,title_2,abstract_2,other_applications_2
2,CA3076363A1,CA3076363A1,CA3076363A,Filling system suitable for solid drag reducin...,no_data,2020-03-20,Petrochina Co Ltd;,Ying Xiong;Youquan Liu;Yadong Zhang;Yuan Xu;Xi...,2023-07-04,E21B21/062,...,patent number,applicant2,authors2,ipc2,priority2,original2,publication2,title_2,abstract_2,other_applications_2
3,CN208812341U,CN208812341U,CN201821579889.3U,Feeding mechanism in plastic film manufacturin...,CN201821579889.3U;,2018-09-27,Zhejiang Hengcheng Polymer Material Co Ltd;,周永良;赵芬芳;谷定浩;顾斌;应雅明;卿亚平;周惠良;,2019-05-03,no_data,...,patent number,applicant2,authors2,ipc2,priority2,original2,publication2,title_2,abstract_2,other_applications_2
4,WO2009102955A2,US20090202702A1,"US12/370,991",Apparatus and method for mixing a powder with ...,"US12/370,991;",2009-02-13,MOOK;,Tamir Farber;Boaz Zimmerman;,2009-08-13,B01F21/20,...,patent number,applicant2,authors2,ipc2,priority2,original2,publication2,title_2,abstract_2,other_applications_2
5,CN219522752U,CN219522752U,CN202320755927.0U,Feeding mechanism for plastic particle processing,CN202320755927.0U;,2023-04-07,Langfang Feizhong New Materials Co ltd;,任春强;李桂飞;董鹏飞;,2023-08-15,Y02W30/62,...,patent number,applicant2,authors2,ipc2,priority2,original2,publication2,title_2,abstract_2,other_applications_2
6,CN111074714B,US11406951B2,"US17/029,057",Continuous and forced asphalt mixing productio...,no_data,2020-09-23,Shanghai Shishen Industry and Trade Co Ltd;,Guangsheng Shi;Zhaowu Shi;Qingzhao Shi;,2022-08-09,E01C7/22,...,patent number,applicant2,authors2,ipc2,priority2,original2,publication2,title_2,abstract_2,other_applications_2
7,US8215930B2,US8215930B2,"US12/261,325",Diaphragm pumps and transporting drag reducers,"US12/261,325;",2008-10-30,LiquidPower Specialty Products Inc;,Timothy L. Burden;Dung H. Nguyen;Richard D. Th...,2012-07-10,F04B43/067,...,patent number,applicant2,authors2,ipc2,priority2,original2,publication2,title_2,abstract_2,other_applications_2
8,CN110015736B,CN110015736B,CN201910367942.6A,Modular likepowder flocculation medicament rat...,CN201910367942.6A;,2019-05-05,Anhui University of Science and Technology;,周伟;冯岸岸;朱金波;张勇;闵凡飞;蔡川川;,2021-10-29,B01F33/8212,...,patent number,applicant2,authors2,ipc2,priority2,original2,publication2,title_2,abstract_2,other_applications_2
9,JPH08266880A,US5634715A,"US08/608,002",Installation for the mixing of liquid and soli...,no_data,1996-03-04,Buehler GmbH;,Norbert Stehr;Dieter Marz;,1997-06-03,B01F33/25,...,patent number,applicant2,authors2,ipc2,priority2,original2,publication2,title_2,abstract_2,other_applications_2


In [43]:
final_data.to_csv('final_data.csv')

In [44]:
files.download('/content/saved_data.txt')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

# tests part
